In [1]:
#r "../src/NMoneys/bin/Debug/net6.0/NMoneys.dll"

using NMoneys;
using NMoneys.Support;
using System.Numerics;
using System.Reflection;
using System.Globalization;

Microsoft.DotNet.Interactive.Formatting.Formatter.ListExpansionLimit= 200;


record Diff(string Name, string FromCurrency, string FromGlobalization)
{
	private static string str<T>(T i) where T : IBinaryInteger<T> 
	{
		return i.ToString("D", CultureInfo.InvariantCulture);
	}
	private static string str<T>(T[] i) where T : IBinaryInteger<T> 
	{
		return string.Join('|', i);
	}

	public static Diff[] Build(Currency currency)
	{
		CanonicalCultureAttribute attr = currency.GetCanonicalCulture();
		if (attr is null) return Array.Empty<Diff>();

		CultureInfo ci = attr.Culture();
		NumberFormatInfo nfi = ci.NumberFormat;
		var ri = new RegionInfo(ci.Name);
		StringComparer ordinal = StringComparer.Ordinal;
		StringComparer native = StringComparer.Create(ci, false);

		List<Diff> diffs = new List<Diff>(10);
		if (currency.IsoCode != Currency.Code.Parse(ri.ISOCurrencySymbol))
		{
			diffs.Add(new Diff("ri.ISOCurrencySymbol", currency.AlphabeticCode, ri.ISOCurrencySymbol));
		}
		if (!ordinal.Equals(currency.EnglishName, ri.CurrencyEnglishName))
		{
			diffs.Add(new Diff("ri.CurrencyEnglishName", currency.EnglishName, ri.CurrencyEnglishName));
		}
		if (!native.Equals(currency.NativeName, ri.CurrencyNativeName))
		{
			diffs.Add(new Diff("ri.CurrencyNativeName", currency.NativeName, ri.CurrencyNativeName));
		}
		if (!native.Equals(currency.Symbol, ri.CurrencySymbol))
		{
			diffs.Add(new Diff("ri.CurrencySymbol", currency.Symbol, ri.CurrencySymbol));
		}
		if (currency.SignificantDecimalDigits != nfi.CurrencyDecimalDigits)
		{
			diffs.Add(new Diff("nfi.CurrencyDecimalDigits", str(currency.SignificantDecimalDigits), str(nfi.CurrencyDecimalDigits)));
		}
		if (!ordinal.Equals(currency.DecimalSeparator, nfi.CurrencyDecimalSeparator))
		{
			diffs.Add(new Diff("nfi.CurrencyDecimalSeparator", currency.DecimalSeparator, nfi.CurrencyDecimalSeparator));
		}
		if (!ordinal.Equals(currency.GroupSeparator, nfi.CurrencyGroupSeparator))
		{
			diffs.Add(new Diff("nfi.CurrencyGroupSeparator", currency.GroupSeparator, nfi.CurrencyGroupSeparator));
		}
		if (!currency.GroupSizes.SequenceEqual(nfi.CurrencyGroupSizes.Select(s => (byte)s)))
		{
			diffs.Add(new Diff("nfi.CurrencyGroupSizes", str(currency.GroupSizes), str(nfi.CurrencyGroupSizes)));
		}
		if (currency.PositivePattern != nfi.CurrencyPositivePattern)
		{
			diffs.Add(new Diff("nfi.CurrencyPositivePattern", str(currency.PositivePattern), str(nfi.CurrencyPositivePattern)));
		}
		if (currency.NegativePattern != nfi.CurrencyNegativePattern)
		{
			diffs.Add(new Diff("nfi.CurrencyNegativePattern", str(currency.NegativePattern), str(nfi.CurrencyNegativePattern)));
		}

		return diffs.ToArray();
	}
};

# Explained discrepancies

* `ALL`: English name ("Lek") consistent with ISO list, symbol ("L") consistent with Wikipedia
* `ARS`: Native name ("Peso") consistent with Wikipedia to remove confusion with previous ARP
* `BDT`: English name ("Taka") consistent with ISO list, native name ("টাকা") consistent with Wikipedia
* `AMD`: Native name ("դրամ") consistent with Wikipedia
* `BBD`: English name ("Barbados Dollar") consistent with ISO list
* `BMD`: English name ("Bermudian Dollar") consistent with ISO list and the native name
* `BTN`: English name ("Ngultrum") consistent with ISO list
* `BOB`: English name ("Boliviano") consistent with ISO list
* `BWP`: English name ("Pula") consistent with ISO list and native name

In [2]:
Currency.FindAll()
	.Select(c => new { Currency = c, Diffs = Diff.Build(c) })
	.Where(a => a.Diffs.Length  > 0)


index value 0 { Currency = ALL, Diffs = Submission#2+Diff[] } Currency ALL IsoCode ALL NumericCode 8 PaddedNumericCode 008 EnglishName Lek Symbol L AlphabeticCode ALL IsoSymbol ALL SignificantDecimalDigits 0 MinAmount 1 NativeName Leku shqiptar DecimalSeparator , GroupSeparator   GroupSizes [ 3 ] NegativePattern 8 PositivePattern 3 FormatInfo System.Globalization.NumberFormatInfo CurrencyDecimalDigits 0 CurrencyDecimalSeparator , IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator   CurrencySymbol L NaNSymbol NaN CurrencyNegativePattern 8 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 0 NumberDecimalSeparator , NumberGroupSeparator   CurrencyPositivePattern 3 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None IsObsolete False Entity <null> Diffs index value 0 Diff { Name = ri.CurrencyEnglishName, FromCurrency = Lek, FromGlobalization = Albanian Lek } Name ri.CurrencyEnglishName FromCurrency Lek FromGlobalization Albanian Lek 1 Diff { Name = ri.CurrencySymbol, FromCurrency = L, FromGlobalization = Lekë } Name ri.CurrencySymbol FromCurrency L FromGlobalization Lekë 1 { Currency = ARS, Diffs = Submission#2+Diff[] } Currency ARS IsoCode ARS NumericCode 32 PaddedNumericCode 032 EnglishName Argentine Peso Symbol $ AlphabeticCode ARS IsoSymbol ARS SignificantDecimalDigits 2 MinAmount 0.01 NativeName Peso DecimalSeparator , GroupSeparator . GroupSizes [ 3 ] NegativePattern 9 PositivePattern 2 FormatInfo System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator , IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator . CurrencySymbol $ NaNSymbol NaN CurrencyNegativePattern 9 NumberNegativePattern 2 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator , NumberGroupSeparator . CurrencyPositivePattern 2 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None IsObsolete False Entity CharacterReference { CodePoint = 36, Name = dollar, DecimalReference = &#36;, HexadecimalReference = &#x24;, NamedReference = &dollar;, Character = $ } CodePoint 36 Name dollar DecimalReference &#36; HexadecimalReference &#x24; NamedReference &dollar; Character $ Diffs index value 0 Diff { Name = ri.CurrencyNativeName, FromCurrency = Peso, FromGlobalization = peso argentino } Name ri.CurrencyNativeName FromCurrency Peso FromGlobalization peso argentino 2 { Currency = BDT, Diffs = Submission#2+Diff[] } Currency BDT IsoCode BDT NumericCode 50 PaddedNumericCode 050 EnglishName Taka Symbol ৳ AlphabeticCode BDT IsoSymbol BDT SignificantDecimalDigits 2 MinAmount 0.01 NativeName টাকা DecimalSeparator . GroupSeparator , GroupSizes [ 3, 2 ] NegativePattern 5 PositivePattern 1 FormatInfo System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator . IsReadOnly True CurrencyGroupSizes [ 3, 2 ] NumberGroupSizes [ 3, 2 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator , CurrencySymbol ৳ NaNSymbol NaN CurrencyNegativePattern 5 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator . NumberGroupSeparator , CurrencyPositivePattern 1 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None IsObsolete False Entity CharacterRefere